# CISC/CMPE 452/COGS 400 Assignment 3 - Unsupervised Learning (10 points)  

Please put your name and student id

    Joseph Moraru, #20181593

- Make sure to run all the cells from the beginning before submission. Do not clear out the outputs. You will only get credit for code that has been run.
- Mark will be deducted based on late policy (-1% of the course total marks per day after due date until the end date after which no assignments will be accepted).
- You can only use Numpy to build the models. Other packages such as Pandas, Sklearn and Scipy can be used for evaluation metrics calculating, data processing, and file reading and writing.

### Files need to be uploaded for this assignment: A3.ipynb, output.wav, and output.csv

## Part 1 Principle Component Analysis Network (5 points)
The dataset "data/sound.csv" contains two sounds recorded by the two microphones. The goal of this assignment is using PCA network to find the approximation of the first principal component.
- Build a PCA network (refer to Principal Component Analysis slide #22 and #23) to reduce the number of features from 2 to 1 (3 points)  
- Train the model and generate the processed data (1 point)  
- Save the data into output.wav and output.csv files (1 point)  
- Compare the sound_o.wav (audio with noise) and output.wav (audio is denoised)  

In [1]:
import numpy as np
import pandas as pd
from scipy.io import wavfile

In [2]:
samrate = 8000

In [3]:
# read csv into array
txtData = np.genfromtxt('data/sound.csv', delimiter=',')
txtData.shape

(50000, 2)

In [4]:
# save array to WAV file
scaledData = np.int16(txtData * samrate)
wavfile.write('data/sound_o.wav', samrate, scaledData)

In [5]:
# read WAV file into array
# The data in sound.csv is processed
# If you use the data generated here, you need to process the data by adding wavData = wavData / samrate
samrate, wavData = wavfile.read('data/sound_o.wav')
wavData = wavData / samrate
samrate, wavData.shape

(8000, (50000, 2))

In [6]:
# save array to csv file
np.savetxt('data/sound_o.csv', txtData, delimiter=',')

In [7]:
# build PCA model and only Numpy can be used
class PCA(object):
    def __init__(self, lr, epoch):
        #lr: learning rate
        #epoch: the number of iterations
        self.lr = lr
        self.epoch = epoch

    def train(self, x, n_components):
        #train the pca model
        #x: input data
        #n_components: the number of components
        
        '''
        #Statistical Solution: (like Q3 in Theory)
        #it is a stasticial approach to PCA as outlined in
        #the beginning of the PCA lecture. I calculate the 
        #coveriance matrix, determine the eigenvalues and eigenvectors,
        #sort them, and then select the top n_components eigenvectors
        #finally I return the transformed data as the the dot product of the normalized x and the selected eigenvectors

        
        #first calculate the mean of the data
        #x = x - np.mean(x, axis=0)
        x_mean = np.mean(x, axis=0)
        x_std = np.std(x, axis=0)
        x_normalized = (x - x_mean) / x_std
        
        #calculate the covariance matrix
        cov_matrix = np.cov(x_normalized.T)

        #caluclate the eigenvalues and eigenvectors
        eig_values, eig_vectors = np.linalg.eig(cov_matrix)
        
        #sort the eigenvalues in decending order (largest eigenvalues are more valuable to us)
        #switching to -eig_value here changes which part of the audio is isolated
        #right now it set to isolating the music portion of the audio
        eig_values_desc = np.argsort(-1*eig_values)[::-1] 
        
        #sort the eigenvectors according to the sorted eigenvalues
        eig_vectors_sorted = eig_vectors[:,eig_values_desc]
        
        #select the top n_components eigenvectors
        select_eig_vectors = eig_vectors_sorted[:,:n_components]

        #return the transformed data
        # normally this would be just -> np.dot(x_normalized, select_eig_vectors)
        # I multiplied by 0.25 to make the audio quieter (it was clipping before)
        return np.dot(x_normalized, select_eig_vectors) * 0.25
        '''
        
        #ANN Solution:
        #it is a neural network approach to PCA as outlined in
        #the end of the PCA lecture.

        #first initiaalize some random weights
        weights = np.random.rand(x.shape[1])

        #loop through the number of epochs
        for _ in range(self.epoch):
            #loop through the row vectors of x
            for row_v in x:
                #calculate the ouput, y as W * x
                y = np.dot(weights, row_v)
                #calculate K as y^2
                K = y * y
                #update the weights by the formula:
                #dW = (lr * y * x) - (K * w)
                weights += self.lr * np.dot(y, row_v) - np.dot(K, weights)
        
        #multiplied by a factor of 4 to make the audio louder
        return x.dot(weights) * 4



In [8]:
# initialize and train the model
pca = PCA(lr=0.1, epoch=15)
#change number of features from 2 to 1 (n_components)
transformed_data = pca.train(wavData, n_components=1)
print(transformed_data)

[-1.71303955 -0.09886383  0.04177317 ...  0.46135556  0.44728224
  0.356279  ]


In [141]:
#save the Data to output csv file
np.savetxt('data/output.csv', transformed_data, delimiter=',')
#save the Data to output wav file
scaledData = np.int16(transformed_data * samrate)
wavfile.write('data/output.wav', samrate, scaledData)

## Part 2 K-Means Clustering Algorithm (5 points)
The dataset is [Palmer Archipelago (Antarctica) penguin data](https://www.kaggle.com/datasets/parulpandey/palmer-archipelago-antarctica-penguin-data) which has 6 features and 1 label called species (Chinstrap, Adélie, or Gentoo)  
The dataset is saved in the "data/penguins_size.csv" file and preprocessed into x_train, x_test, y_train, y_test  
- Build a K-Means clustering algorithm (refer to Unsupervised Learning slide #29) to cluster the preprocessed data (2 points)  
- Standardize the data and train the model with the training set (1 point)  
- Evaluate the model and print the confusion matrixes with both training and test sets (2 points)  


In [142]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [143]:
# load the dataset
data = pd.read_csv('data/penguins_size.csv')
data.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [144]:
# data preprocessing
data = data.dropna()
data = data[data['sex'] != '.']

cleanup_nums = {"species": {"Adelie": 0, "Chinstrap": 1, "Gentoo": 2},
                "island": {"Biscoe": 0, "Dream": 1, "Torgersen": 2},
                "sex": {"MALE": 0.0, "FEMALE": 1.0}}
data = data.replace(cleanup_nums)

data.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,2,39.1,18.7,181.0,3750.0,0.0
1,0,2,39.5,17.4,186.0,3800.0,1.0
2,0,2,40.3,18.0,195.0,3250.0,1.0
4,0,2,36.7,19.3,193.0,3450.0,1.0
5,0,2,39.3,20.6,190.0,3650.0,0.0


In [145]:
x = np.array(data.drop(['species'], axis=1).copy())
y = np.array(data['species'].copy()).astype(int)

In [146]:
# data standardization
from sklearn.preprocessing import StandardScaler

print("Original Data:")
print("Mean:", np.mean(x, axis=0))
print("Standard Deviation:", np.std(x, axis=0))
print()
x = StandardScaler().fit_transform(x)
print("Normalized Data:")
print("Mean:", np.mean(x, axis=0))
print("Standard Deviation:", np.std(x, axis=0))

Original Data:
Mean: [6.51651652e-01 4.39927928e+01 1.71648649e+01 2.00966967e+02
 4.20705706e+03 4.95495495e-01]
Standard Deviation: [7.13641408e-01 5.46045096e+00 1.96627643e+00 1.39947048e+01
 8.04005860e+02 4.99979709e-01]

Normalized Data:
Mean: [ 4.26752394e-17  3.84077154e-16  6.40128591e-16  2.13376197e-16
 -1.70700958e-16 -2.46716228e-17]
Standard Deviation: [1. 1. 1. 1. 1. 1.]


In [147]:
# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((266, 6), (67, 6), (266,), (67,))

In [148]:
# calculate the confusion matrix
from sklearn.metrics import confusion_matrix

def evaluator(train, pred): 
    train_confusion_matrix = confusion_matrix(train, pred)
    print('Confusion matrix:\n', train_confusion_matrix)

In [149]:
# setup a baseline model
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3) # n_clusters - the number of clusters
km.fit(x_train)
y_pred = km.predict(x_train)
evaluator(y_train, y_pred)
y_pred = km.predict(x_test)
evaluator(y_test, y_pred)

Confusion matrix:
 [[ 60   0  47]
 [ 32   0  26]
 [  0 101   0]]
Confusion matrix:
 [[13  0 26]
 [ 2  0  8]
 [ 0 18  0]]


In [150]:
# build K-means model and only Numpy can be used
class KMeans(object):
    def __init__(self, n_clusters, random_state=0, centroids=None):
        self.n_clusters = n_clusters
        self.random_state = random_state
        self.centroids = centroids

    def train(self, x, learning_rate=0.1, n_iters=100):
        #deleted : y, x_test, y_test, from parameters, as in my implmentation they are not needed
        
        #set a random seed for reproducibility if needed (works without this)
        if self.random_state != None:
            np.random.seed(self.random_state)
        
        #randomly select n clusters centroids
        self.centroids = x[np.random.choice(x.shape[0], self.n_clusters, replace=False)]
        
        #loop through n times
        for n in range(n_iters):
            #find the euclidean distance between each point and each centroid as a np array
            #below is same as:
            #e_dist = (((x - self.centroids[0])**2).sum(axis=1))**0.5
            e_dist = np.linalg.norm(x[:, np.newaxis] - self.centroids, axis=2)
           
            #find the closest centroid for each point
            #closest = np.argmin(e_dist, axis=0)
            closest = np.argmin(e_dist, axis=1)
            
            #loop through each cluster
            for cluster_idx in range(self.n_clusters):
                #find the new centroid for each cluster
                #perform the mean of the points in the cluster where closest == cluster_idx in x
                moved_centroids = x[closest == cluster_idx].mean(axis=0)

                #for any points where closest == cluster_idx, move the centroid closer to the points
                if np.any(closest == cluster_idx):
                    #centroids act as weights in this case
                    self.centroids[closest] += learning_rate * (moved_centroids - self.centroids[closest])
                else:
                    #centroids act as weights in this case
                    self.centroids[closest] -= learning_rate * (moved_centroids - self.centroids[closest])
        

    def predict(self, x):
        #same as:
        #e_dist = (((x - self.centroids[0])**2).sum(axis=1))**0.5
        #closest = np.argmin(e_dist, axis=0)
        e_dist = np.linalg.norm(x[:, np.newaxis] - self.centroids, axis=2)
        closest = np.argmin(e_dist, axis=1)
        return closest


In [151]:
# initialize and train the model
my_km = KMeans(n_clusters=3, random_state=5)
my_km.train(x_train, learning_rate=0.01, n_iters=100)
y_pred = my_km.predict(x_train)
y_pred_test = my_km.predict(x_test)

In [152]:
# evaluate the model and print the confusion matrixes for both training and test sets
evaluator(y_train, y_pred)
print()
evaluator(y_test, y_pred_test)

Confusion matrix:
 [[ 94   2  11]
 [ 12  24  22]
 [  0   0 101]]

Confusion matrix:
 [[36  0  3]
 [ 7  1  2]
 [ 0  0 18]]
